# Computation Theory

In [1]:
#imports

import numpy as np

### Problem 1: Binary Words and Operations

# Parity Function Documentation

---

### Overview
The **parity function** is one of the logical functions used in the SHA-1 algorithm. It takes in three 32-bit words and compares their bits to decide the result for each position. Parity looks at each bit of the three inputs and checks if an **odd number of them are 1**. If the number of 1s is odd, the result bit is 1; if it's even, the result bit is 0.

### How It Works
The parity function works by applying the **XOR operator** to each bit of the three inputs (`x`, `y`, and `z`). XOR returns `1` if the number of `1` bits is **odd**, and `0` if its number of bits is **even**.

For exmaple, if we look at a single bit position across `x`, `y`, and `z`:

| x | y | z | Result (x ⊕ y ⊕ z) |
|:-:|:-:|:-:|:-----------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

You can see from the table that the result is `1` whenever **an odd number of inputs are 1**. 

In Python, this can be done simply by XORing the three values:
``x ^ y ^ z``

When using 32-bit unsigned integers, this operation is applied to **all 32 bits at once**, producing a new 32-bit result.

You can also conclude from the table that the order of either the `1` or `0` bits is not significant in the outcome. This means that the parity function is **commutative**.

### Why It's Used

The parity function is used in **specific rounds of the SHA-1 algorithm** to combine three different 32-bit values in a balanced way. XOR is useful because it dosen't favour any single input - the output changes if **any one** of the inputs changes.

This properly helps SHA-1 achieve **good diffusion**, meaning small changes in the input data quickly spread through the algorithm and affect many bits of the final hash.

Parity is also very simple to calculate, which makes it efficient to use repeatedly during the hashing process without slowing things down.

### Function Signature

Below is the Python implementation of the **parity function**. It takes three 32-bit integers (`x`, `y`, and `z`) and returns their bitwise XOR as a 32-bit unsigned integer. This matches the behavior defined in the SHA-1 standard:

```python
import numpy as np

def Parity(x: int, y: int, z: int) -> np.uint32:

In [4]:
def Parity(x,y,z):
    """
    Computes the parity (bitwise XOR) of three 32-bit unsigned integers.

    This function takes three integer inputs (x, y, z), converts them into
    32-bit unsigned integers to match the behavior expected in the SHA-1
    algorithm, and then applies the XOR operation across all 32 bits of
    each input. The result is a single 32-bit unsigned integer where each
    bit represents the parity of the corresponding bits of x, y, and z.

    The parity is 1 if an odd number of the three bits are 1, and 0 if an
    even number of the three bits are 1. This operation is commutative
    and matches the definition used in the Secure Hash Standard (FIPS 180-4).

    Args:
        x (int): First integer input.
        y (int): Second integer input.
        z (int): Third integer input.

    Returns:
        np.uint32: The bitwise XOR (parity) of x, y, and z as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32(x ^ y ^ z)

### Testing the Parity Function

To make sure the `Parity` function works correctly, we can test it with a few simple inputs where we can easily calculate the expected result by hand.

For example:

- If all three inputs are `0`, the result should be `0` because no bits are set.  
- If one input is `1` and the others are `0`, the result should be `1`.  
- If two inputs are `1`, the result should be `0` (because 2 is even).  
- If all three inputs are `1`, the result should be `1` (because 3 is odd).

### Parity Function Test Cases:
---

In [36]:
test_cases = [
    (0, 0, 0),  # Expected: 0
    (1, 0, 0),  # Expected: 1
    (1, 1, 0),  # Expected: 0
    (1, 1, 1),  # Expected: 1
    (0b1010, 0b1100, 0b0110),  # Expected 0
    (0xFFFFFFFF, 0x00000000, 0xFFFFFFFF),  # Expected: 0 
]

for x, y, z in test_cases:
    result = Parity(x,y,z)
    print(f"Parity({x}, {y}, {z}) = {result:04b}")

Parity(0, 0, 0) = 0000
Parity(1, 0, 0) = 0001
Parity(1, 1, 0) = 0000
Parity(1, 1, 1) = 0001
Parity(10, 12, 6) = 0000
Parity(4294967295, 0, 4294967295) = 0000


### Expected Results From Parity Function Tests

- `Parity(0, 0, 0)` → `0`  
- `Parity(1, 0, 0)` → `1`  
- `Parity(1, 1, 0)` → `0`  
- `Parity(1, 1, 1)` → `1`  
- `Parity(0b1010, 0b1100, 0b0110)` → `0`  
- `Parity(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF)` → `0`

These tests confirm that the function behaves exactly as expected, matching the bitwise XOR logic defined in the SHA-1 specification.


## Choose Function Documentation

---

### Overview
The **choose function** (often written as `Ch`) is another logical function used in SHA-1.  
It takes three 32-bit words: `x`, `y`, and `z`.  
For each bit position, it **chooses** between `y` and `z` based on the bit of `x`:

- If the bit of `x` is **1**, the result takes the bit from **`y`**.
- If the bit of `x` is **0**, the result takes the bit from **`z`**.

Because of this, people say: **“`x` chooses between `y` and `z`.”**

### How It Works

Bit-by-bit, the choose function is defined as:

`Ch(x, y, z) = (x AND y) XOR ((NOT x) AND z)`

This formula matches the “choose” idea because of how the bitwise operations work:

- When a bit of **x is 1**, the expression `(x AND y)` keeps the bit from **y** in that position  
  At the same time, `(~x AND z)` becomes `0 AND z` for that bit (because NOT 1 = 0), so the bit from **z** is ignored there.

- When a bit of **x is 0**, `(x AND y)` becomes `0 AND y` → which is always 0.  
  Meanwhile, `(~x AND z)` becomes `1 AND z` (because NOT 0 = 1), so the bit from **z** is kept in that position.

- The two parts are then joined together with XOR, but since **only one of the two expressions can produce a 1 for any bit**, XOR here is effectively just combining them without any conflict.

So, for each bit:
- **x = 1** → choose the bit from **y**.  
- **x = 0** → choose the bit from **z**.

This is why it’s called the “choose” function — `x` decides, bit by bit, whether to pick the value from `y` or from `z`.

**Example Table:**
| x | y | z | Ch(x, y, z) |
|:-:|:-:|:-:|:------------:|
| 0 | 0 | 0 |      0       |
| 0 | 0 | 1 |      1       | 
| 0 | 1 | 0 |      0       |
| 0 | 1 | 1 |      1       |
| 1 | 0 | 0 |      0       | 
| 1 | 0 | 1 |      0       |
| 1 | 1 | 0 |      1       |
| 1 | 1 | 1 |      1       |

You can see the pattern:  
- When **x=0**, the output equals **z**.  
- When **x=1**, the output equals **y**.

### Why It’s Used
The choose function helps SHA-1 **mix data in a controlled way**:
- It lets one value (`x`) **select** bits from two other values (`y` and `z`).
- A **single bit change** in `x` can flip which source is chosen at many positions, helping with **diffusion** (small input changes affect many output bits).
- It is also **cheap to compute** on hardware and in code (only AND, NOT, XOR).

### Function Signature

```python
import numpy as np

def Choose(x: int, y: int, z: int) -> np.uint32:


In [30]:
def Choose(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers.
    Chooses bits from y and z based on the bits of x.
    If a bit in x is 1, the corresponding bit from y is chosen.
    If a bit in x is 0, the corresponding bit from z is chosen.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        32-bit unsigned integer: result of the expression (x & y) ^ (~x & y)

    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ ((~x) & z))

### Choose Function Test Case:
---

In [35]:
test_cases = [
    (0, 0, 0),  # x=0, choose z, 0
    (0, 0, 1),  # x=0, choose z, 1
    (0, 1, 0),  # x=0, choose z, 0
    (0, 1, 1),  # x=0, choose z, 1

    (1, 0, 0),  # x=1, choose y, 0
    (1, 0, 1),  # x=1, choose y, 0
    (1, 1, 0),  # x=1, choose y, 1
    (1, 1, 1),  # x=1, choose y, 1

    (0b0101, 0b1111, 0b0000),  # choose y at x=1 bits, 0b0101
    (0b1010, 0b1111, 0b0000),  # choose y at x=1 bits, 0b1010
]

for x, y, z in test_cases:
    result = Choose(x, y, z)
    print(f"x={x}, y={y}, z={z} = result={result:04b}")


x=0, y=0, z=0 = result=0000
x=0, y=0, z=1 = result=0001
x=0, y=1, z=0 = result=0000
x=0, y=1, z=1 = result=0001
x=1, y=0, z=0 = result=0000
x=1, y=0, z=1 = result=0000
x=1, y=1, z=0 = result=0001
x=1, y=1, z=1 = result=0001
x=5, y=15, z=0 = result=0101
x=10, y=15, z=0 = result=1010


### Expected Results for Choose Function Tests

- (0, 0, 0) = 0  
- (0, 0, 1) = 1  
- (0, 1, 0) = 0  
- (0, 1, 1) = 1  

- (1, 0, 0) = 0  
- (1, 0, 1) = 0  
- (1, 1, 0) = 1  
- (1, 1, 1) = 1  

- (0b0101, 0b1111, 0b0000) = 0b0101  
- (0b1010, 0b1111, 0b0000) = 0b1010  

These results match the expected behavior of the **Choose** function:
- When `x = 0`, the result equals `z`.
- When `x = 1`, the result equals `y`.
- When `x` is mixed, the function chooses bits from `y` where `x` is 1 and from `z` where `x` is 0.


In [16]:
def Majority(x,y,z):
    """
    function takes in three parameters x,y,z (integers).
    Converts them to 32-bit unsigned integers.
    Returns the majority value among the three inputs.

    Args:
        x (int): first integer input
        y (int): second integer input
        z (int): third integer input

    Returns:
        32-bit unsigned integer: result of the expression (x & y) ^ (x & z) ^ (y & z) 
        
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ (x & z) ^ (y & z))

Sigma Functions

In [17]:
def Sigma0(x):
    """
    function takes in one paramater (integer x)
    Converts x to a 32-bit unsigned integer
    returns Big Sigma O by using its formula

    Args: 
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x))

    """
    x = np.uint32(x)
    return np.uint32(
        ((x >> 2) | (x << (32 - 2))) ^
        ((x >> 13) | (x << (32 - 13))) ^
        ((x >> 22) | (x << (32 - 22)))
    )

def Sigma1(x):
    """
    function takes in one parameter (integer x)
    Converts x to a 32-bit unsigned integer
    returns Big Sigma 1 by using its formula

    Args: 
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x))

    """
    x = np.uint32(x)

    return np.uint32(
    ((x >> 6) | (x << (32 - 6))) ^
    ((x >> 11) | (x << (32 - 11))) ^
    ((x >> 25) | (x << (32 - 25)))
    )

def sigma0(x):
    """
    function takes in one parameter (integer x)
    Converts x to a 32-bit unsigned integer
    returns Small Sigma 0 by using its formula

    Args: 
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x))

    """
    x = np.uint32(x)

    return(
        ((x >> 7) | x << ((32 - 7))) ^
        ((x >> 18) | x << ((32 - 18))) ^
        ((x >> 3))
    )

def small_sigma1(x):
    """

    function takes in one parameter (integer x)
    Converts x to a 32-bit unsigned integer
    returns Small Sigma 1 by using its formula

    Args:
        x (int): only int value

    Return:
        a 32-bit unsigned integer: result of (ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x))
    """
    
    x = np.uint32(x)
    return np.uint32(
        ((x >> 17) | (x << (32 - 17))) ^
        ((x >> 19) | (x << (32 - 19))) ^
        (x >> 10)
    )
    


### Problem 2: Fractional Parts of Cube Roots

### Problem 3: Padding

### Problem 4: Hashes

### Problem 5: Passwords